In [1]:
import json
import sqlite3
import pandas as pd
import seaborn as sns

In [2]:
conn = sqlite3.connect('./retrieve_data/data_sanitization/data/articles_with_author_mapping.db')

In [3]:
c = conn.cursor()

In [4]:
def encode_abbr(abbr):
    full = '[\"%' + '%'.join(abbr) + '%\"]'
    left = '[\"%' + '%'.join(abbr) + '%\",%]'
    right = '[%,\"%' + '%'.join(abbr) + '%\"]'
    return (full, left, right)

abbr = "jhz"

In [5]:
# todo 08.07
# befor ich mir die ganze mphe mache hier, vielleicht mal schauen, wie viel prozent der autoren ich durch das einfach certainty und bisschen frequncy mapping abdecken kann
# und wo es dann konflikte gibt

In [6]:
c.execute("select a.author_array, min(a.published_at), max(a.published_at), count(*) as c from articles a where a.author_array in (select '[\"' || abbreviation || '\"]' from authors a group by abbreviation  order by count(*) desc) group by a.author_array order by c desc limit 20")
authors = [{"name": row[0], "min_published": row[1], "max_published": row[2]} for row in c.fetchall()]

In [19]:
for author in authors:
    print((encode_abbr(author['name'][2:-2])[0], author["name"], author["min_published"], author["max_published"], ))
    c.execute("select author_array, min_p, max_p from (select a.author_array, min(a.published_at) as min_p, max(a.published_at) as max_p, count(*) as c from articles a where a.author_array LIKE ? and a.author_array not like '%,%' and a.author_array != ? group by a.author_array order by c desc) where min_p > DATE(?,'-6 months') and max_p < DATE(?,'+6 months') limit 3", (encode_abbr(author['name'][2:-2])[0], author["name"], author["min_published"], author["max_published"], ))
    matches = c.fetchall()
    print(f"{len(matches)} articles found")
    print(matches)

('["%L%V%Z%"]', '["LVZ"]', '2010-01-06T07:11:22+00:00', '2021-12-17T14:15:00+00:00')
10 articles found
[('["lvz"]', '2011-03-02T19:09:19+00:00', '2021-12-31T11:00:04+00:00'), ('["Olivia Jasmin Czok"]', '2013-10-10T23:00:00+00:00', '2015-02-08T22:30:00+00:00'), ('["LVZ Familie"]', '2020-08-12T12:56:00+00:00', '2020-12-28T11:37:00+00:00'), ('["Ulrike von Leszczynski"]', '2014-12-25T11:01:47+00:00', '2019-06-25T12:51:34+00:00'), ('["fLVZ"]', '2016-02-14T10:39:00+00:00', '2017-10-24T09:39:00+00:00'), ('["LVZ Redaktion"]', '2021-07-08T05:01:51+00:00', '2021-08-01T15:35:39+00:00'), ('["joka LVZ-Online"]', '2014-07-16T11:15:11+00:00', '2014-07-16T11:34:00+00:00'), ('["Live-Ticker zum"]', '2016-01-30T19:18:00+00:00', '2016-02-26T15:46:00+00:00'), ('["Leipziger Volkszeitung"]', '2018-04-14T03:00:00+00:00', '2018-10-27T03:04:00+00:00'), ('["LVZ-Redaktion"]', '2020-05-31T19:29:24+00:00', '2021-02-04T08:26:00+00:00')]
('["%j%o%k%a%"]', '["joka"]', '2010-02-23T18:53:36+00:00', '2021-03-23T16:08:04+

In [ ]:
df = pd.DataFrame(columns=["id", "url", "context_tag", "organization", "author_array", "author_is_abbreviation", "published_at"], data=articles)

In [ ]:
df.loc[df['author_array'] == '["' + abbr.upper() + '"]', 'author_array'] = '["' + abbr.lower() + '"]'

In [ ]:
# aggregate by quarter of the year
df['published_at'] = pd.to_datetime(df['published_at'])
df['quarter'] = df['published_at'].dt.quarter
df['year'] = df['published_at'].dt.year
df['year_quarter'] = df['year'].astype(str) + "/" + df['quarter'].astype(str)


In [ ]:
top_authors = df.groupby('author_array').count()['id'].sort_values(ascending=False).head(4)
top_authors

In [ ]:
# get top 5 authors with most articles
top_authors = df.groupby('author_array').count()['id'].sort_values(ascending=False).head(4).index.tolist()
top_authors = top_authors + ['["' + abbr + '"]'] if '["' + abbr + '"]' not in top_authors else top_authors
# filter for top authors
df_top_authors = df[df['author_array'].isin(top_authors)].groupby(['year_quarter', 'author_array']).count()['id'].reset_index()


In [ ]:

sns.relplot(kind='line', data=df_top_authors, x='year_quarter', y='id', hue='author_array', errorbar=None)
sns.set(rc={'figure.figsize':(14,10)})
#df.groupby('year').count()['id'].plot(kind='bar')

